#Final code

In [72]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

In [73]:
# Para burlar o bloqueador de webscraping
headers = {
  'User-Agent' :
	'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:125.0) Gecko/20100101 Firefox/125.0'

}

url = 'https://www.fragrantica.com.br/notas/'

In [74]:
response = requests.get(url, headers=headers)

In [75]:
soup = BeautifulSoup(response.content, 'html.parser')

In [76]:
# Criar listas vazias para armazenar os dados
links = []
img_alts = []

# Encontrar todos os elementos <a> com a classe específica
links_with_class = soup.find_all('a')

# Iterar sobre os links encontrados
for link in links_with_class:
    # Extrair o href do link
    link_href = link['href']

    # Verificar se o link começa com "https://www.fragrantica.com.br/notas/"
    if link_href.startswith("https://www.fragrantica.com.br/notas/"):
        # Encontrar o elemento <img> dentro do link
        img_tag = link.find('img')

        # Se houver um elemento <img>, extrair o valor do atributo alt
        if img_tag:
            img_alt = img_tag.get('alt')
        else:
            img_alt = None

        # Adicionar o link e o valor do atributo alt às listas
        links.append(link_href)
        img_alts.append(img_alt)

# Criar DataFrame com os dados coletados
df = pd.DataFrame({'Link': links, 'Img Alt': img_alts})

In [78]:
def find_elements(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    n = 1
    not_found_count = 0
    results = []

    while not_found_count < 2:
        selector = f'.medium-order-2 > div:nth-child(1) > div:nth-child({n}) > div:nth-child(1) > div:nth-child(1) > div:nth-child(3) > h3:nth-child(1) > a:nth-child(1)'
        elements = soup.select(selector)

        if elements:
            for element in elements:
                results.append(element.text.strip())
            n += 1
        else:

            not_found_count += 1
            n += 1

        if not_found_count == 2:
            #print(f"No matching elements found for selector: {selector}")
            break

    return results

In [79]:
# Function to get data from each link and extract elements
def process_link(link):
    headers = {"User-Agent": "Your User Agent"}  # Set your user agent
    response = requests.get(link, headers=headers)

    if response.status_code == 200:
        found_elements = find_elements(response.content)
        return found_elements
    else:
        print(f"Failed to retrieve data from {link}. Status code: {response.status_code}")
        return []

In [83]:
# Faça uma cópia do DataFrame original
df_subset = df.copy()

# Criar uma nova lista para armazenar os elementos processados
processed_elements = []

# Aplicar a função process_link apenas nos primeiros 5 links do DataFrame
for link in tqdm(df_subset['Link'], desc="Processing links"):
    processed_elements.append(process_link(link))

# Atualizar o DataFrame com os elementos processados
df_subset['Elements'] = processed_elements

Processing links: 100%|██████████| 1488/1488 [2:29:49<00:00,  6.04s/it]


In [84]:
df_subset

,Link,Img Alt,Elements
0,https://www.fragrantica.com.br/notas/Agua-Citr...,Água Cítrica,"[Britney Spears Fantasy Sheer, Kensie Buttercu..."
1,https://www.fragrantica.com.br/notas/Bergamota...,Bergamota,"[Dior Dior Homme Cologne 2022, Monotheme Venez..."
2,https://www.fragrantica.com.br/notas/Calamondi...,Calamondin,"[Bvlgari Bvlgari Man Terrae Essence, Molinard ..."
3,https://www.fragrantica.com.br/notas/Candied-L...,Candied Lemon,[Dolce&Gabbana Devotion]
4,https://www.fragrantica.com.br/notas/Capim-Lim...,Capim-Limão ou Citronela,"[Louis Vuitton Cactus Garden, Aquaflor Firenze..."
...,...,...,...
1483,https://www.fragrantica.com.br/notas/Veneno-13...,Veneno,"[Água de Cheiro Absinto Poison, Police Police ..."
1484,https://www.fragrantica.com.br/notas/Verdox-10...,Verdox,"[Corps Volatils Blessed Fruit, Alexandra Gluck..."
1485,https://www.fragrantica.com.br/notas/Vinil-499...,Vinil,"[Demeter Fragrance Vinyl, UERMI XX ± Latex, Xy..."
1486,https://www.fragrantica.com.br/notas/Vinyl-Gua...,Vinyl Guaiacol,[Etat Libre d'Orange The Ghost In The Shell]


In [85]:
df_subset.to_csv('perfumes.csv')

In [94]:
# Criando uma nova lista para armazenar os dicionários
data = []

# Iterar sobre cada linha e extrair elementos
for i, row in tqdm(df_subset.iterrows(), total=len(df_subset), desc="Processando links"):
    link = row['Img Alt']
    elements = row['Elements']
    for element in elements:
        data.append({'Img Alt': link, 'Element': element})

# Criando um novo DataFrame a partir da lista de dicionários
new_df = pd.DataFrame(data)


Processando links: 100%|██████████| 1488/1488 [00:00<00:00, 8227.96it/s]


In [ ]:
new_df

In [127]:
# Criar dummy variables sem adicionar o sufixo "_dummy"
dummies = pd.get_dummies(new_df['Img Alt'], prefix='', prefix_sep='')


In [125]:
dummies

,Pepperwood™_dummy,Abacate_dummy,Abacaxi_dummy,Abeto Balsâmico Canadense_dummy,Abeto Negro_dummy,Abio_dummy,Abre Caminho_dummy,Abricó-de-Praia_dummy,Abrunheiro_dummy,Absinto_dummy,...,Árvore-do-Viajante_dummy,Árvore-do-caril_dummy,Árvores_dummy,Âmbar_dummy,Íris_dummy,Íris Palida_dummy,Óleo da Árvore do Chá_dummy,Óleo de Motor_dummy,Ópio_dummy,Óxido de Rosa_dummy
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69675,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
69676,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
69677,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
69678,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [128]:
new_df2 = new_df.copy()

# Agrupar os dados pelo 'Element'
grupos = new_df.groupby('Element')

# Iterar sobre os grupos e preencher as colunas dummy
for elemento, grupo in grupos:
    # Verificar se há correspondência no outro DataFrame
    if elemento in new_df2.columns:
        # Obter os valores correspondentes do outro DataFrame
        valores_correspondentes = new_df2[elemento].values
        # Preencher as colunas dummy com os valores correspondentes
        dummies.loc[grupo.index] = valores_correspondentes.reshape(-1, 1)

# Adicionar os nomes das colunas dummy
dummies.columns = [f'{col}' for col in dummies.columns]

# Concatenar as dummy variables com o DataFrame original
new_df2 = pd.concat([new_df, dummies], axis=1)

In [131]:
new_df2.drop(columns='Img Alt', inplace=True)

In [136]:
# Agrupar os dados por elemento e aplicar a função any() para cada grupo
new_df2_grouped = new_df2.groupby('Element').any().reset_index()

# As colunas que são todas falsas permanecerão como falsas,
# enquanto as que têm pelo menos um verdadeiro permanecerão como verdadeiras.

# Se você deseja manter o índice original, você pode fazer assim:
new_df2_grouped.set_index('Element', inplace=True)

# Se quiser redefinir o índice, pode fazer assim:
new_df2_grouped.reset_index(inplace=True)

In [137]:
elemento_procurado = "Britney Spears Fantasy Sheer"

# Selecionar a linha correspondente ao elemento
linha_elemento = new_df2_grouped[new_df2_grouped['Element'] == elemento_procurado]

# Filtrar as colunas onde o valor é True
colunas_verdadeiras = linha_elemento.iloc[:, 1:].columns[linha_elemento.iloc[:, 1:].values.flatten()]

print("Colunas com valor True para o elemento", elemento_procurado, ":")
colunas_verdadeiras


Colunas com valor True para o elemento Britney Spears Fantasy Sheer :


Index(['Cupcake', 'Lichia', 'Marmelo', 'Vitória Régia', 'Água Cítrica'], dtype='object')

In [141]:
new_df2_grouped

,Element,Pepperwood™,Abacate,Abacaxi,Abeto Balsâmico Canadense,Abeto Negro,Abio,Abre Caminho,Abricó-de-Praia,Abrunheiro,...,Árvore-do-Viajante,Árvore-do-caril,Árvores,Âmbar,Íris,Íris Palida,Óleo da Árvore do Chá,Óleo de Motor,Ópio,Óxido de Rosa
0,#Parfum Hashtag #CrazyGirl,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,#Parfum Hashtag #Dream4ever,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,#Parfum Hashtag #iloveYou,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,-DIS DIS-1 (Decay),False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,-DIS DIS-2 (Concrete),False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40243,南柯记 Nankeji Rustle 瑟瑟,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
40244,南柯记 Nankeji Spring Cherry Blossom 春之樱,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
40245,南柯记 Nankeji Summer Poem 夏之诗,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
40246,南柯记 Nankeji Winter Wine 冬之酒,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [139]:
# Verificar se a coluna "Água Cítrica" é verdadeira para cada elemento
elementos_com_agua_citrica_verdadeira = new_df2_grouped[new_df2_grouped['Água Cítrica']]

# Exibir os elementos que possuem a coluna "Água Cítrica" verdadeira
print("Elementos com a coluna 'Água Cítrica' verdadeira:")
elementos_com_agua_citrica_verdadeira


Elementos com a coluna 'Água Cítrica' verdadeira:


,Element,Pepperwood™,Abacate,Abacaxi,Abeto Balsâmico Canadense,Abeto Negro,Abio,Abre Caminho,Abricó-de-Praia,Abrunheiro,...,Árvore-do-Viajante,Árvore-do-caril,Árvores,Âmbar,Íris,Íris Palida,Óleo da Árvore do Chá,Óleo de Motor,Ópio,Óxido de Rosa
5498,Bath & Body Works Indigo Dreams,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6930,Britney Spears Fantasy Sheer,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
20016,Kensie Buttercup Babe,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
35265,The Body Shop Lemon Hair Mist,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [148]:
new_df2_grouped = new_df2_grouped.replace({False: 0, True: 1})

In [149]:
new_df2_grouped

,Element,Pepperwood™,Abacate,Abacaxi,Abeto Balsâmico Canadense,Abeto Negro,Abio,Abre Caminho,Abricó-de-Praia,Abrunheiro,...,Árvore-do-Viajante,Árvore-do-caril,Árvores,Âmbar,Íris,Íris Palida,Óleo da Árvore do Chá,Óleo de Motor,Ópio,Óxido de Rosa
0,#Parfum Hashtag #CrazyGirl,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,#Parfum Hashtag #Dream4ever,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,#Parfum Hashtag #iloveYou,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-DIS DIS-1 (Decay),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-DIS DIS-2 (Concrete),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40243,南柯记 Nankeji Rustle 瑟瑟,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40244,南柯记 Nankeji Spring Cherry Blossom 春之樱,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40245,南柯记 Nankeji Summer Poem 夏之诗,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40246,南柯记 Nankeji Winter Wine 冬之酒,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [140]:
new_df2_grouped.to_csv('perfumes_completo.csv')

# Clusterização

In [150]:
from sklearn.cluster import KMeans

In [159]:
teste = new_df2_grouped.copy()

In [ ]:
# Selecionar todas as colunas exceto a coluna 'Element'
dados_clusterizacao = teste.drop(columns=['Element'])

# Escolher o número de clusters
numero_clusters = 100

# Inicializar o modelo KMeans
kmeans = KMeans(n_clusters=numero_clusters)

# Treinar o modelo nos dados
kmeans.fit(dados_clusterizacao)

# Adicionar os rótulos dos clusters de volta ao DataFrame original
teste['cluster'] = kmeans.labels_

In [ ]:
teste

# Extra code

In [142]:
url = 'https://www.fragrantica.com.br/notas/Agua-Citrica-1182.html'
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

links2 = soup.find_all('a')


In [143]:
# Using CSS Selector
elements = soup.select('.medium-order-2 > div:nth-child(1) > div:nth-child(n) > div:nth-child(1) > div:nth-child(1) > div:nth-child(3) > h3:nth-child(1) > a:nth-child(1)')

                        #.medium-order-2 > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > div:nth-child(1) > div:nth-child(3) > h3:nth-child(1) > a:nth-child(1)

# Printing the text content of the first matched element
if elements:
    print(elements[0].text.strip())
else:
    print("No matching elements found.")

Britney Spears Fantasy Sheer


In [144]:
# Find all <a> tags with class "link-span" inside <a> tags with attribute "perfume"
perfume_links = soup.find_all('a', attrs={'href': True})

# Extracting the desired parts
for link in perfume_links:
    print(link.text.strip())




Clube
Notícias
Perfumes
Pesquisa
Designers
Países
Empresas do Grupo
Setores
Notas
Perfumistas
Pesquisa por Notas
Buscador de Perfumes
Grupos
Cores
Awards
Awards 2023
Awards 2022
Awards 2021
Awards 2020
Awards 2019
Awards 2018
Awards 2017
Sobre nós
Notas
Perfumistas
Clube
Sobre nós
Pesquisa por Notas
Britney Spears Fantasy Sheer
Kensie Buttercup Babe
Bath & Body Works Indigo Dreams
The Body Shop Lemon Hair Mist
Logar
Registrar
Tom Ford Black Orchid
JAFRA Gardenia Blossom
Adidas Adidas UEFA Champions League Edition
Hugo Boss Boss Bottled Eau de Parfum
Dolce&Gabbana Light Blue
Tom Ford Tobacco Vanille
Carolina Herrera 212 VIP Rosé
The Body Shop Strawberry
Zara Applejuice
Natura Luna Ousadia
L'acqua Di Fiori Cupcake Me Napolitano Chic
L’Occitane Au Brésil Capim-Limão Maracujá
Mancera Roses & Chocolate
Antonio Banderas Blue Seduction
Paris Elysees La Petite Fleur Blanche
Novas Resenhas
Coleção de Lembranças Perfumadas de Jo Malone London: Silhuetas Desprovidas de Cor e Substância
Perfume